# Import and install libraries

In [ ]:
!pip install -q dwave-ocean-sdk

In [ ]:
!pip install gurobipy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import product
from google.colab import files
from getpass import getpass
import json
from google.colab import userdata
from collections import Counter
from copy import deepcopy
import time

In [ ]:
from collections import defaultdict
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
import dwave.inspector as inspector
from dimod import ConstrainedQuadraticModel, CQM, SampleSet
from dwave.system import LeapHybridCQMSampler
from dimod.vartypes import Vartype
from dimod import Binary, quicksum
from dimod import BinaryQuadraticModel

In [ ]:
import gurobipy as gp
from gurobipy import GRB

# Setup dwave account and Gurobi Accounts

In [ ]:
endpoint = 'https://cloud.dwavesys.com/sapi'
token = userdata.get('dwave_leap')

In [ ]:
WLSACCESSID = userdata.get('gurobi_WLSACCESSID')
WLSSECRET = userdata.get('gurobi_WLSSECRET')
LICENSEID = int(userdata.get('gurobi_LICENSEID'))

In [ ]:
params = {
"WLSACCESSID": WLSACCESSID,
"WLSSECRET": WLSSECRET,
"LICENSEID": LICENSEID,
}
env = gp.Env(params=params)

# Utility Functions

In [ ]:
def augment_distance_matrix(original_c):
    N = original_c.shape[0]  # Original number of cities
    augmented_c = np.zeros((N + 1, N + 1))  # Initialize augmented matrix with zeros

    # Copy the original distances into the augmented matrix
    augmented_c[1:, 1:] = original_c

    # Since the depot node is virtual, distances from and to the depot are zero
    # The distances from node 0 to other nodes and vice versa are already zero

    return augmented_c

In [ ]:
def generate_setup_matrix(N, min_setup, max_setup, symmetric=True,seed = 101):
    # Initialize the matrix with zeros on the diagonal
    matrix = np.zeros((N, N), dtype=int)

    # Fill the upper triangle with random setup times
    np.random.seed(seed)
    for i in range(N):
        for j in range(i + 1, N):
            setup_time = np.random.randint(min_setup, max_setup + 1)
            matrix[i, j] = setup_time
            if symmetric:
                matrix[j, i] = setup_time  # Ensure symmetry by copying to the lower triangle
            else:
                matrix[j, i] = np.random.randint(min_setup, max_setup + 1)  # Different setup time if asymmetric

    return matrix

In [ ]:
def generate_durations(N,min_dur,max_dur,seed = 101):

  np.random.seed(seed)
  durations = np.random.randint(min_dur, max_dur, size=(N))

  return durations

In [ ]:
class PSMP:

  def __init__(self,n_jobs,n_machines,symmetric,seed = 101,min_setup_dur =1,max_setup_dur = 10, min_duration = 1, max_duration = 20):
    self.n_jobs = n_jobs
    self.n_machines = n_machines
    self.symmetric = symmetric
    self.seed = seed
    self.min_setup_dur = min_setup_dur
    self.max_setup_dur = max_setup_dur
    self.min_duration = min_duration
    self.max_duration = max_duration

    self.N = list(range(1, n_jobs + 1))
    self.N0 = [0] + self.N
    self.M = list(range(1, n_machines + 1))

    self.s =  augment_distance_matrix(generate_setup_matrix(n_jobs,self.min_setup_dur,self.max_setup_dur,self.symmetric,self.seed))
    self.d = np.array([0] +  generate_durations(n_jobs,self.min_duration,self.max_duration,self.seed).tolist())

    self.W = (self.n_jobs//self.n_machines) + 2

  def generate_initial_assignments(self):
    # Initialize an empty list for each machine

    JOBS = np.array(self.N)

    machines_durations = []
    machines = []

    for i in self.N:
      machines.append(list(set(self.N) - set([i])))

    for mac_assign in machines:
      machine_duration = 0
      for i in range(len(mac_assign) - 1):
        current_act = mac_assign[i]
        next_act = mac_assign[i + 1]
        machine_duration += self.s[current_act][next_act]
      machine_duration += self.d[mac_assign].sum()
      machines_durations.append(machine_duration)

    initial_schedules = dict()
    for m,mac_assign in enumerate(machines):
      initial_schedules[m] = {'route':mac_assign,'cost':machines_durations[m]}

    return initial_schedules

  def calculate_makespan_from_sequence(self,sequence):

    makespan = 0
    for i in range(len(sequence) - 1):
      current_act =sequence[i]
      next_act = sequence[i + 1]
      makespan += self.s[current_act][next_act]
    makespan += self.d[sequence].sum()

    return makespan

In [ ]:
class PSMP_2:
    def __init__(self, file_path=None, seed=101, min_setup_dur=1, max_setup_dur=10, min_duration=1, max_duration=20):
        # Check if file path is provided
        if file_path:
            # Read and parse the input file
            with open(file_path, "r") as file:
                # Remove empty lines and strip extra spaces
                lines = [line.strip() for line in file if line.strip()]

            # Parse the values
            self.n_jobs = int(lines[0])  # Number of jobs
            self.n_machines = int(lines[1])  # Number of machines
            self.symmetric = lines[2] == "True"  # Symmetry flag

            # Job durations (4th line, split into space-separated values)
            self.d = np.array([int(x) for x in lines[3].split()])
            # Add a dummy zero for job 0
            self.d = np.array([0] + self.d.tolist())

            # Setup times (remaining lines)
            self.s = np.array([[int(x) for x in row.split()] for row in lines[4:]])

        else:
            raise ValueError("No file path provided.")

        # Initialize the rest of the attributes

        self.N = list(range(1, self.n_jobs + 1))
        self.N0 = [0] + self.N
        self.M = list(range(1, self.n_machines + 1))

        self.W = (self.n_jobs // self.n_machines) + 2

    def generate_initial_assignments(self):
        # Initialize an empty list for each machine
        JOBS = np.array(self.N)

        machines_durations = []
        machines = []

        for i in self.N:
            machines.append(list(set(self.N) - set([i])))

        for mac_assign in machines:
            machine_duration = 0
            for i in range(len(mac_assign) - 1):
                current_act = mac_assign[i]
                next_act = mac_assign[i + 1]
                machine_duration += self.s[current_act][next_act]
            machine_duration += self.d[mac_assign].sum()
            machines_durations.append(machine_duration)

        initial_schedules = dict()
        for m, mac_assign in enumerate(machines):
            initial_schedules[m] = {'route': mac_assign, 'cost': machines_durations[m]}

        return initial_schedules

    def calculate_makespan_from_sequence(self, sequence):
        makespan = 0
        for i in range(len(sequence) - 1):
            current_act = sequence[i]
            next_act = sequence[i + 1]
            makespan += self.s[current_act][next_act]
        makespan += self.d[sequence].sum()

        return makespan

In [ ]:
def determine_feasible(solution):

    tour_vars = {key: value for key, value in solution.items() if key.startswith('x') and value == 1}

    # Step 2: Extract positions from the 'x' variables
    positions = {}
    nodes = {}

    for var in tour_vars:
      _, i, p = var.split('_')
      i, p = int(i), int(p)

      # Check if there's already a node assigned to this position (feasibility check)
      if p in positions:
          # raise ValueError(f"Infeasible solution: Multiple nodes occupy position {p}")
          return False
      positions[p] = i

      # Check if the node is assigned to only one position (feasibility check)
      if i in nodes:
          # raise ValueError(f"Infeasible solution: Node {i} occupies multiple positions")
          return False
      nodes[i] = p

    # Step 3: Construct the tour in sequence order
    max_position = max(positions.keys())
    tour = []
    for p in range(max_position + 1):
        if p in positions:
            tour.append(positions[p])
        else:
            # raise ValueError(f"Incomplete solution: Missing node at position {p}")
            return False
    return True

In [ ]:
def parse_tsp_solution(PSMP_class, solution ,duals):
    # Step 1: Gather only the 'x' variables with value 1
    s = PSMP_class.s
    d = PSMP_class.d
    tour_vars = {key: value for key, value in solution.items() if key.startswith('x') and value == 1}

    # Step 2: Extract positions from the 'x' variables
    positions = {}
    nodes = {}

    for var in tour_vars:
        _, i, p = var.split('_')
        i, p = int(i), int(p)

        # Check if there's already a node assigned to this position (feasibility check)
        if p in positions:
            raise ValueError(f"Infeasible solution: Multiple nodes occupy position {p}")
        positions[p] = i

        # Check if the node is assigned to only one position (feasibility check)
        if i in nodes:
            raise ValueError(f"Infeasible solution: Node {i} occupies multiple positions")
        nodes[i] = p

    # Step 3: Construct the tour in sequence order
    max_position = max(positions.keys())
    tour = []
    for p in range(max_position + 1):
        if p in positions:
            tour.append(positions[p])
        else:
            raise ValueError(f"Incomplete solution: Missing node at position {p}")

    # Step 4: Calculate the cost of the tour
    tour_with_depot = [-1] + tour + [-1]  # Add depot at start and end
    route = []

    for i in range(len(tour_with_depot) - 1):
        current_node = tour_with_depot[i]
        next_node = tour_with_depot[i + 1]

        route.append((current_node, next_node))

    duals_cost = 0
    tour_cost = 0
    for v,u in route:
      if v == -1 or u == -1:
        setup = 0
      else:
        setup = s[v,u]
      tour_cost += setup

      if u == -1:
        dual = 0
        dur = 0
      else:
        dur = d[u]
        dual = duals[u]
      tour_cost += dur
      duals_cost += dual

    return route,tour, tour_cost, duals_cost

In [ ]:
def repair_solution(broken_sample, setup_cost_matrix, Q):

    num_jobs = len(Q)
    num_positions = len(Q)

    broken_solution = []

    for var in broken_sample:
      if broken_sample[var] == 1:
        broken_solution.append(var)

    # Parse the broken solution into dictionaries
    job_to_positions = defaultdict(list)
    position_to_jobs = defaultdict(list)
    for var in broken_solution:
        _, job, position = var.split('_')
        job, position = int(job), int(position)
        job_to_positions[job].append(position)
        position_to_jobs[position].append(job)

    # Initialize repaired solution with placeholders
    repaired_solution = [-1] * num_positions  # -1 indicates empty position

    # Assign each position a single job, resolving conflicts
    for p in range(num_positions):
        if len(position_to_jobs[p]) == 1:  # If exactly one job assigned to this position
            repaired_solution[p] = position_to_jobs[p][0]
        elif len(position_to_jobs[p]) > 1:  # If multiple jobs assigned, resolve conflict
            previous_job = repaired_solution[p - 1] if p > 0 else None
            next_job = None
            # Find the next filled position
            for future_p in range(p + 1, num_positions):
                if repaired_solution[future_p] != -1:
                    next_job = repaired_solution[future_p]
                    break
            # Choose the job with the minimum setup cost
            min_cost = float('inf')
            best_job = None
            for job in position_to_jobs[p]:
                cost = 0
                if previous_job is not None:
                    cost += setup_cost_matrix[previous_job][job]
                if next_job is not None:
                    cost += setup_cost_matrix[job][next_job]
                if cost < min_cost:
                    min_cost = cost
                    best_job = job
            repaired_solution[p] = best_job

    # Fill missing jobs
    assigned_jobs = set(repaired_solution)
    missing_jobs = set(Q) - assigned_jobs
    unassigned_positions = [i for i, pos in enumerate(repaired_solution) if pos == -1]

    # Assign missing jobs to empty positions
    for job in missing_jobs:
        if unassigned_positions:
            repaired_solution[unassigned_positions.pop(0)] = job

    # Final check to resolve duplicates and ensure all jobs in Q are assigned
    assigned_jobs = set(repaired_solution)
    duplicates = [job for job in repaired_solution if repaired_solution.count(job) > 1]
    unassigned_jobs = set(Q) - assigned_jobs

    for i, job in enumerate(repaired_solution):
        if job in duplicates and unassigned_jobs:
            repaired_solution[i] = unassigned_jobs.pop()
            duplicates.remove(job)

    # Ensure every job in Q is in the solution and no duplicates exist
    repaired_solution = [
        job if repaired_solution.count(job) == 1 else -1
        for job in repaired_solution
    ]

    missing_jobs = set(Q) - set(repaired_solution)
    for i, job in enumerate(repaired_solution):
        if job == -1 and missing_jobs:
            repaired_solution[i] = missing_jobs.pop()

    return repaired_solution

# Sub Pricing Clustering function

In [ ]:
def select_core_jobs(PSMP_class, duals, tabu_list, alpha=1, beta=0.5, gamma=0.5, delta=10):
    """
    Select core jobs with a tabu mechanism to avoid repeated selections.

    Args:
        jobs: List of job IDs.
        duals: Dictionary of dual values for each job.
        durations: Dictionary of durations for each job.
        setup_times: 2D matrix or dictionary of setup times between jobs.
        num_machines: Number of machines (number of clusters to form).
        tabu_list: Set of job IDs currently in the tabu list.
        alpha, beta, gamma, delta: Weights for duals, durations, distance, and tabu penalty.

    Returns:
        List of selected core jobs, updated tabu list.
    """

    jobs = PSMP_class.N
    durations = PSMP_class.d
    setup_times = PSMP_class.s
    num_machines = PSMP_class.n_machines

    # Step 1: Sort jobs by adjusted scores
    scores = {}
    for job in jobs:
        # print(job)
        # Calculate distance to closest selected core job
        min_distance = min(setup_times[job][core] for core in tabu_list) if tabu_list else 0

        # Calculate tabu penalty
        tabu_penalty = delta if job in tabu_list else 0

        # Calculate overall score
        scores[job] = alpha * duals[job] + beta * durations[job] - gamma * min_distance - tabu_penalty

    # Sort jobs by score (highest first)
    sorted_jobs = sorted(scores, key=scores.get, reverse=True)

    # Step 2: Select core jobs
    core_jobs = []
    for _ in range(num_machines):
        for job in sorted_jobs:
            if job not in core_jobs:
                core_jobs.append(job)
                break

    return core_jobs

In [ ]:
def assign_balanced_clusters(PSMP_class,core_jobs, duals, alpha=1.0):
    """
    Assign jobs to balanced clusters using np.array_split to determine cluster sizes.

    Args:
        jobs: List of job IDs.
        core_jobs: List of core job IDs, one per machine.
        setup_times: 2D matrix or dictionary of setup times between jobs.
        durations: Dictionary of durations for each job.
        duals: Dictionary of dual values for each job.
        num_machines: Number of clusters/machines.
        alpha: Weight for dual values in the cost function.

    Returns:
        clusters: Dictionary with machine IDs as keys and lists of assigned jobs as values.
    """
    jobs = PSMP_class.N
    durations = PSMP_class.d
    setup_times = PSMP_class.s
    num_machines = PSMP_class.n_machines


    # Determine target sizes for each cluster
    split_jobs = np.array_split(jobs, num_machines)
    target_sizes = [len(split) for split in split_jobs]

    # Initialize clusters with core jobs
    clusters = {m: [core_jobs[m]] for m in range(num_machines)}
    remaining_jobs = set(jobs) - set(core_jobs)  # Jobs yet to be assigned

    # Iteratively assign jobs to meet target sizes
    while remaining_jobs:
        best_cost = float('inf')
        best_job = None
        best_machine = None

        # Evaluate all unassigned jobs for all machines
        for job in remaining_jobs:
            for machine in range(num_machines):
                # Skip machines that have already met their target size
                if len(clusters[machine]) >= target_sizes[machine]:
                    continue

                # Get the last job in the current machine's cluster
                last_job = clusters[machine][-1]

                # Compute the cost of assigning this job to the machine
                cost = setup_times[last_job][job] + durations[job] - alpha * duals[job]

                # Track the best assignment
                if cost < best_cost:
                    best_cost = cost
                    best_job = job
                    best_machine = machine

        # Assign the best job to the best machine
        if best_machine is not None:
            clusters[best_machine].append(best_job)
            remaining_jobs.remove(best_job)
        else:
            raise ValueError("No valid assignment possible. Please check inputs.")

    return clusters

In [ ]:
def refine_clusters_all_jobs_2(clusters, core_jobs, PSMP_class):
    """
    Refine clusters to minimize the maximum makespan by exploring job moves.

    Args:
        clusters: Dictionary of clusters with machine IDs as keys and lists of jobs as values.
        core_jobs: List of core jobs that cannot be moved.
        PSMP_class: Class instance with the method `calculate_makespan_from_sequence(seq)`.

    Returns:
        Refined clusters if improvements were made, otherwise an empty list.
    """
    improved = True
    changes_made = False  # Track if any changes were made

    jobs = PSMP_class.N
    durations = PSMP_class.d
    setup_times = PSMP_class.s
    num_machines = PSMP_class.n_machines

    clusters = deepcopy(clusters)

    while improved:
        improved = False

        # Get current makespans
        makespans = {
            machine: PSMP_class.calculate_makespan_from_sequence(seq)
            for machine, seq in clusters.items()
        }

        max_makespan = max(makespans.values())

        # Iterate over all clusters and jobs
        for source_machine, jobs in clusters.items():
            # Skip core jobs
            movable_jobs = [job for job in jobs if job not in core_jobs]

            for job in movable_jobs:
                # Try moving this job to each target machine
                for target_machine in clusters.keys():
                    if target_machine == source_machine:
                        continue  # Skip moving to the same machine

                    # Perform the move (temporarily)
                    clusters[source_machine].remove(job)
                    clusters[target_machine].append(job)

                    # Recalculate makespans
                    new_makespans = {
                        machine: PSMP_class.calculate_makespan_from_sequence(seq)
                        for machine, seq in clusters.items()
                    }

                    # Check if the max makespan improves
                    new_max_makespan = max(new_makespans.values())
                    if new_max_makespan < max_makespan:
                        # Commit the move
                        max_makespan = new_max_makespan
                        makespans = new_makespans
                        improved = True
                        changes_made = True  # Record that a change was made
                        break  # Exit loop to re-evaluate from updated state
                    else:
                        # Revert the move
                        clusters[target_machine].remove(job)
                        clusters[source_machine].append(job)

                if improved:
                    break  # Re-evaluate from updated state

            if improved:
                break  # Re-evaluate from updated state

    return clusters if changes_made else []

# Sequencing sub-problem

In [ ]:
def sequence_sub_problem(PSMP_instance,cluster,dual_values, num_reads=100, ANNEALTIME=20,silence=True):

  Q = np.array(cluster)
  N = len(Q)
  P = range(N)
  P_m_1 = range(N - 1)

  d = PSMP_instance.d
  s = PSMP_instance.s

  LAMBDA = int(d.sum() + s.sum())
  CHAIN_S = LAMBDA

  bqm = BinaryQuadraticModel(Vartype.BINARY)


  x = np.array([[f'x_{i}_{p}' for p in P] for i in Q])
  if not silence:
    print(x)
    print("="*150)
  for i in Q:
    for j in Q:
        if i != j:
            for p in P_m_1:
              cost = s[i][j]
              nameVar1 = f"x_{i}_{p}"
              nameVar2 = f"x_{j}_{p+1}"
              # print(f"- {i,j}  {cost}.({nameVar1}).({nameVar2})")
              bqm.add_quadratic(nameVar1,nameVar2,cost)

  for i in Q:
      for p in P:
          nameVar = f"x_{i}_{p}"
          # print(f"- {d[i]}.({nameVar})")
          bqm.add_linear(nameVar,1*d[i])

  for p in P:
    terms = x[:,p]
    expr = " - "
    expr = expr + " + ".join(terms)
    expr = expr + " = 1"
    if not silence:
      print(expr)
    terms = [(x,1) for x in terms]
    bqm.add_linear_equality_constraint(terms=terms,lagrange_multiplier=LAMBDA,constant=-1)
    terms.clear
  if not silence:
    print("="*150)
  for i in Q:
    terms = [f"x_{i}_{p}" for p in P]
    expr = " - "
    expr = expr + " + ".join(terms)
    expr = expr + " == 1"
    if not silence:
      print(expr)
    terms = [(x,1) for x in terms]
    bqm.add_linear_equality_constraint(terms=terms,lagrange_multiplier=LAMBDA,constant=-1)
    terms.clear

  if not silence:
    print("="*150)
    print("Running QA")


  inst = f"w-sequence_sub"

  sampler = DWaveSampler(solver='Advantage_system6.4',endpoint= endpoint, token = token)
  if not silence:
    print("Connected to sampler", sampler.solver.name)

  sampler = EmbeddingComposite(sampler)
  sampler_name = sampler.properties['child_properties']['chip_id']

  sampleset_2 = sampler.sample(bqm, num_reads= 100 ,annealing_time=ANNEALTIME,label=inst,chain_strength=CHAIN_S)
  time = sampleset_2.info['timing']['qpu_access_time']/1e6

  feasible_solutions_subproblem_2 = []
  not_feasible_solutions_subproblem_2 = []
  for i,sample in enumerate(sampleset_2):
    fea = determine_feasible(sample)
    if fea:
      route,tour, tour_cost, duals_cost = parse_tsp_solution(PSMP_instance,sample,dual_values)
      if tour not in feasible_solutions_subproblem_2:
        feasible_solutions_subproblem_2.append(tour)
        # print(i,fea)
    else:
      not_feasible_solutions_subproblem_2.append(sample)

  repaired_feasible = []
  for i,brok in enumerate(not_feasible_solutions_subproblem_2):
    repaired = repair_solution(brok,s,Q)
    if len(set(Q) - set(repaired)) > 0:
      # print(i,repaired,len(repaired),set(Q) - set(repaired))
      continue
    else:
      if repaired not in repaired_feasible:
        repaired_feasible.append(repaired)

  total_seqs = feasible_solutions_subproblem_2 + repaired_feasible
  return total_seqs,time

# Reading an instance from File

In [ ]:
PSMP_instance =  PSMP_2('10_2_False.pmsp')

# Build Master Problem



In [ ]:
initial_sequences = PSMP_instance.generate_initial_assignments()

In [ ]:
master = gp.Model("MasterProblem_PMSP",env=env)

lambda_vars = {}
for idx in range(len(initial_sequences)):
    lambda_vars[idx] = master.addVar(vtype=GRB.INTEGER, name=f"col_{idx}") # Column variables

C_max = master.addVar(vtype=GRB.INTEGER, name="C_max") # C_max variable

# Objective: Minimize makespan
master.setObjective(C_max, GRB.MINIMIZE)

# Constraints: Each job must be assigned one time
for job in PSMP_instance.N:
    master.addConstr(
        gp.quicksum(lambda_vars[idx] for idx in lambda_vars if job in initial_sequences[idx]['route']) >= 1,
        name=f"job_{job}"
    )

master.addConstr(gp.quicksum(lambda_vars[idx] for idx in lambda_vars) == PSMP_instance.n_machines, name="number_machines") # Columns = number of machines

for idx in lambda_vars:
  master.addConstr(C_max >= initial_sequences[idx]['cost'] * lambda_vars[idx], name=f"C_max_{idx}") # C_max must be bigger than the cost of the machine

master.update()